In [22]:
from PIL import Image
from torch.utils.data import Dataset

base_path = "../leapGestRecog/"
people = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"]
gestures = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
gestures_map = {
    1: "palm", 
    2: "l", 
    3: "fist", 
    4: "fist_moved",
    5: "thumb", 
    6: "index", 
    7: "ok", 
    8: "palm_moved", 
    9: "c",
    10: "down"
}

class GestureDataset(Dataset):
  def __init__(self, transform = None, target_transform = None):
    img_labels = []
    for person in range(10):
      for label in range(1, 11):
        path = "%s/%s_%s/" % (str(person).zfill(2), str(label).zfill(2), gestures_map[label])
        for id in range(1, 201):
          filename = "frame_%s_%s_%s.png" % (str(person).zfill(2), str(label).zfill(2), str(id).zfill(4))
          img_labels.append((base_path + path + filename, label))
          
    self.img_labels = img_labels 
    self.transform = transform
    self.target_transform = target_transform
  def __getitem__(self, index):
    path, label = self.img_labels[index]
    img = Image.open(path).convert('1') # convert to black and white
    if self.transform is not None:
      img = self.transform(img) 
    return img, label
  def __len__(self):
    return len(self.img_labels)

In [23]:
from torchvision import transforms
from torch.utils.data import DataLoader

gesture_dataset = GestureDataset(transform=transforms.ToTensor())

train_dataloader = DataLoader(gesture_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [24]:
for image, label in train_dataloader:
  print(image.shape)
  print(label)
  break

torch.Size([64, 3, 240, 640])
tensor([ 3,  3,  7,  6,  3,  5,  6,  4,  9,  2,  3, 10,  1,  8, 10,  1,  5,  3,
         1,  1,  4,  4,  1,  2,  8,  8,  1,  5,  2,  2,  6,  7,  7, 10,  6,  4,
         7,  9,  7, 10,  7,  5,  5,  1,  2,  2, 10, 10,  9,  6,  5,  1,  2,  2,
         4,  5,  3,  4,  8,  4,  9,  6,  2,  9])
